In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import altair as alt
import matplotlib.dates as mdates
import numpy as np
from datetime import datetime
print("Done")

Done


In [2]:
def convert_combined(sub_df):
    
    # transpose the dataframe
    sub_df_T = sub_df.T
      
    # rename the first row to 'Value'
    # sub_df_T = sub_df_T.rename(columns={1: 'Value'})
      
    # remove the first row
    sub_df_T = sub_df_T.iloc[1:]
    # add the name to the first column
    # sub_df_T = sub_df_T.rename_axis('Date').reset_index()
      
    date = sub_df_T.index.values
    value = sub_df_T.iloc[:, 0].values
    combined_df = pd.DataFrame({"Date": date, "Value": value})
      
    # convert the date to datetime
    combined_df = combined_df.astype({'Date': 'datetime64[ns]'})

    # check if the first value contains a comma
    if ',' in str(combined_df['Value'].iloc[0]):
        # convert the value to float (remove comma and convert to float)
        combined_df['Value'] = combined_df['Value'].str.replace(',', '')
        # in case some rows have letter in the value
        combined_df['Value']= combined_df['Value'].str.replace(r'[a-zA-Z]', '', regex=True)
        combined_df['Value'] = combined_df['Value'].astype(float)
    else:
        # convert the value to float
        combined_df = combined_df.astype({'Value': 'float'})

    return combined_df

In [3]:
def convert_monthly(sub_df):
    
    # resamples the input DataFrame on the 'Date' column at monthly frequency using the resample method
    # also calculates the mean value for each month    
    monthly_data = sub_df.resample('M', on='Date').mean()
    
    # resets the index of the resulting DataFrame using the reset_index method
    monthly_data = monthly_data.reset_index()
    
    # converts the dates in the 'Date' column of the monthly_data DataFrame to the first day of the corresponding month
    monthly_data['Date'] = monthly_data['Date'].values.astype('datetime64[M]')
    
    return monthly_data

In [4]:
# Define a function to expand a single date from 'YY-MMM' to 'YYYY-MM-01'

def expand_date(date_str):
    date_obj = datetime.strptime(date_str, '%y-%b').replace(day=1)
    return date_obj.strftime('%Y-%m-%d')

In [5]:
# Define a function that does what 'convert_combined' does with not all the steps

def my_transform(some_df):
    some_df_T = some_df.T
    date = some_df_T.index.values
    value = some_df_T.iloc[:, 0].values
    this_df = pd.DataFrame({"Date": date, "Value": value})
    this_df.drop(0, inplace=True) # drop top row
    return this_df

In [6]:
# Define a function that finds correlation between between two columns of a dataframe

def find_correlation(df, col1, col2):
    return df[col1].corr(df[col2])

## RTLBCI

Frequency: Weekly

Table: 33-10-0398-01

Release date: 2023-04-28 RTLBCI

The csv file is gathered from: https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3310039801

In [7]:
file_path = 'Clean/rtlcbi.csv'
RT_LBCI_df = pd.read_csv(file_path)
RT_LBCI_df.head()

,City_Provience,"August 10, 2020","August 17, 2020","August 24, 2020","August 31, 2020","September 7, 2020","September 14, 2020","September 21, 2020","September 28, 2020","October 5, 2020",...,"February 13, 2023","February 20, 2023","February 27, 2023","March 6, 2023","March 13, 2023","March 20, 2023","March 27, 2023","April 3, 2023","April 10, 2023","April 17, 2023"
0,"St. John's, Newfoundland (792) 1",100.00,111.81,122.82,115.91,118.19,149.29,139.92,194.67,223.12,...,241.55,186.13,361.69,338.63,294.84,352.09,311.52,477.58,547.03,606.62
1,"Halifax, Nova Scotia (0348) 1",100.00,103.25,103.77,92.11,111.06,107.50,101.47,110.83,126.49,...,163.05,160.01,227.82,186.08,180.62,218.93,211.18,200.39,189.60,218.11
2,"Moncton, New Brunswick (0539) 1",100.00,108.05,120.78,113.63,130.73,134.45,133.52,175.36,191.65,...,361.62,370.05,428.51,375.67,347.27,427.92,399.65,416.02,420.79,442.13
3,"Quebec, Quebec (0685) 1",100.00,107.75,124.25,103.23,127.96,114.26,80.63,97.85,95.72,...,250.23,291.60,359.60,347.10,281.01,398.41,373.35,360.89,331.55,281.41
4,"Trois-Rivieres, Quebec (0953) 1",100.00,106.45,114.34,99.36,117.63,113.74,101.25,120.78,128.54,...,314.81,332.48,379.26,315.77,346.72,406.18,366.51,370.99,367.63,384.46


In [8]:
# Take halifax as an example
RT_LBCI_Halifax = RT_LBCI_df.iloc[1:2]
RT_LBCI_Halifax = convert_combined(RT_LBCI_Halifax)
RT_LBCI_Halifax.head()

,Date,Value
0,2020-08-10,100.00
1,2020-08-17,103.25
2,2020-08-24,103.77
3,2020-08-31,92.11
4,2020-09-07,111.06


In [9]:
# Montreal
m = RT_LBCI_df.iloc[6:7]
m = convert_combined(m)
m.head()

,Date,Value
0,2020-08-10,100.00
1,2020-08-17,89.20
2,2020-08-24,87.22
3,2020-08-31,98.07
4,2020-09-07,102.95


In [10]:
# Montreal to monthly
RT_LBCI_Montreal_monthly = convert_monthly(m)
RT_LBCI_Montreal_monthly.head(10)

,Date,Value
0,2020-08-01,93.6225
1,2020-09-01,107.2300
2,2020-10-01,120.5750
3,2020-11-01,143.8020
4,2020-12-01,155.1450
5,2021-01-01,163.4400
6,2021-02-01,154.6900
7,2021-03-01,187.1800
8,2021-04-01,216.3425
9,2021-05-01,201.2080


### Weekly

In [11]:
alt.Chart(RT_LBCI_Halifax).mark_line().encode(
    x='Date',
    y='Value'
).interactive()

alt.Chart(...)

Since the RT-LBCI is weekly data, we will convert them into monthly data.
Add four weeks and divide by 4 to get the monthly data.

### Monthly

In [12]:
RT_LBCI_Halifax_monthly = convert_monthly(RT_LBCI_Halifax)
RT_LBCI_Halifax_monthly.head(10)

,Date,Value
0,2020-08-01,99.7825
1,2020-09-01,107.7150
2,2020-10-01,130.6950
3,2020-11-01,150.7020
4,2020-12-01,167.8600
5,2021-01-01,153.1375
6,2021-02-01,111.2725
7,2021-03-01,136.9800
8,2021-04-01,129.5775
9,2021-05-01,175.3300


In [13]:
alt.Chart(RT_LBCI_Halifax_monthly, title = "Halifax Real-time Local Business Condition Index").mark_line().encode(
    x='Date',
    y='Value'
).interactive()

alt.Chart(...)

# Population

Frequency: Monthly

Geography: CMA

Release Date: April 21, 2023

Age: 15 years and over

Both sexes

Total Retail Sales

Time Period: Aug 2020-March 2023

Table: 14-10-0378-01

The csv file is gathered from: 

In [14]:
pop_df = pd.read_csv('Clean/population.csv')
pop_df.head()

,City_Province,20-Aug,20-Sep,20-Oct,20-Nov,20-Dec,21-Jan,21-Feb,21-Mar,21-Apr,...,22-Jun,22-Jul,22-Aug,22-Sep,22-Oct,22-Nov,22-Dec,23-Jan,23-Feb,23-Mar
0,"St. John's, Newfoundland and Labrador",181,181.1,181.2,181.3,181.4,181.5,181.6,181.6,181.7,...,184.1,184.5,184.8,185.2,185.5,185.9,186.2,186.6,187.1,187.6
1,"Halifax, Nova Scotia",373.3,374,374.8,375.4,375.9,376.4,376.8,377.3,377.7,...,391.3,392.6,394.1,395.7,397.2,398.4,399.4,400.4,401.5,402.9
2,"Moncton, New Brunswick",132.1,132.3,132.5,132.6,132.8,132.9,133,133.1,133.3,...,137.6,138.1,138.6,139.1,139.7,140.1,140.6,141,141.5,142
3,"Saint John, New Brunswick",109.9,109.9,110,110,110,110.1,110.1,110.1,110.2,...,112.3,112.6,112.9,113.2,113.5,113.8,114,114.2,114.4,114.7
4,"Saguenay, Quebec",135.7,135.7,135.7,135.7,135.7,135.7,135.7,135.6,135.6,...,135.4,135.4,135.5,135.5,135.5,135.6,135.6,135.6,135.7,135.8


In [15]:
# we will take Halifax as an example
halifax_pop = pop_df.iloc[1:2]
halifax_pop

,City_Province,20-Aug,20-Sep,20-Oct,20-Nov,20-Dec,21-Jan,21-Feb,21-Mar,21-Apr,...,22-Jun,22-Jul,22-Aug,22-Sep,22-Oct,22-Nov,22-Dec,23-Jan,23-Feb,23-Mar
1,"Halifax, Nova Scotia",373.3,374,374.8,375.4,375.9,376.4,376.8,377.3,377.7,...,391.3,392.6,394.1,395.7,397.2,398.4,399.4,400.4,401.5,402.9


In [16]:
pop_halifax = my_transform(halifax_pop)
pop_halifax.head()

,Date,Value
1,20-Aug,373.3
2,20-Sep,374
3,20-Oct,374.8
4,20-Nov,375.4
5,20-Dec,375.9


In [17]:
pop_halifax['Date'] = pop_halifax['Date'].apply(expand_date)
pop_halifax.head()

,Date,Value
1,2020-08-01,373.3
2,2020-09-01,374
3,2020-10-01,374.8
4,2020-11-01,375.4
5,2020-12-01,375.9


In [32]:
alt.Chart(pop_halifax).mark_line().encode(
    x='Date:T',
    y='Value:Q'
).interactive()

alt.Chart(...)

### Merge Population and RT-LBCI

In [19]:
pop_halifax['Date'] = pd.to_datetime(pop_halifax['Date']) # convert date to datetime64[ns] 
mydff = pd.merge(RT_LBCI_Halifax_monthly, pop_halifax, on='Date')

#rename columns
mydff = mydff.rename(columns={'Value_x': 'RT_LBCI_Halifax_monthly', 'Value_y': 'Population'})
mydff['Population'] = mydff['Population'].values.astype(np.float64)
mydff.head()

,Date,RT_LBCI_Halifax_monthly,Population
0,2020-08-01,99.7825,373.3
1,2020-09-01,107.7150,374.0
2,2020-10-01,130.6950,374.8
3,2020-11-01,150.7020,375.4
4,2020-12-01,167.8600,375.9


In [35]:
# combine the plot of the two dataframes
chart111 = alt.Chart(mydff, title = "Halifax Real-time Local Business Condition Index & Population (Monthly)").mark_line().encode(
    x='Date:T',
    y='RT_LBCI_Halifax_monthly:Q',
)

chart222 = alt.Chart(mydff).mark_line(color='red').encode(
    x='Date:T',
    y='Population:Q',
)

(chart111 + chart222
).properties(width=800,height=500
).resolve_scale(y='independent'
).configure_axis(grid=False
).interactive()

alt.LayerChart(...)

### Correlation between RT-LBCI and Population

In [21]:
# drop date
nodate_mydff = mydff.drop(['Date'], axis=1)

# normalize the data
scaled_mydff = (nodate_mydff - nodate_mydff.min()) / (nodate_mydff.max() - nodate_mydff.min())
scaled_mydff.head()

,RT_LBCI_Halifax_monthly,Population
0,0.000000,0.000000
1,0.043525,0.023649
2,0.169614,0.050676
3,0.279390,0.070946
4,0.373534,0.087838


In [22]:
# find correlation
find_correlation(mydff, 'RT_LBCI_Halifax_monthly', 'Population')

0.7540939829656663

There is a strong correlation between Population and RTLBCI.

## Retail trade sales

Frequency: Monthly

Geography: CMA

Adjusted: Seasonally

NAICS: Retail Trade

Release Date: April 21, 2023

Total Retail Sales

Time Period: Aug 2020-May 2023

Table: 20-10-0081-01

The csv file is gathered from: https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=2010008101

Cities:

* Montreal, Quebec
* Toronto, Ontario
* Vancouver, British Columbia

In [23]:
sales_df = pd.read_csv('Clean/retail_sales.csv')
sales_df.head(40)

,City_Province,20-Aug,20-Sep,20-Oct,20-Nov,20-Dec,21-Jan,21-Feb,21-Mar,21-Apr,...,22-May,22-Jun,22-Jul,22-Aug,22-Sep,22-Oct,22-Nov,22-Dec,23-Jan,23-Feb
0,"Montréal, Quebec",5993137,6106531,5972435,6209757,5703922,5092495,6101555,6591588,6671994,...,7011267,6946357,6911373,7114241,6967176,7349779,7239519,7296095,7262899,7302933
1,"Toronto, Ontario",9073104,9513202,9017806,9492266,8955316,9094059,9030230,9641124,8646164,...,11340314,11963393,11009121,11042276,11176432,11181512,11106455,11089462,11066873,11073867
2,"Vancouver, British Columbia",3722961,3921472,3885056,3957390,4006490,4067537,4115896,4129204,4242551,...,4493039,4518670,4470279,4559974,4425744,4461143,4470042,4386798,4517770,4317813


In [24]:
# we will take Montreal as an example
mont_data = sales_df.iloc[0:1]
mont_data

,City_Province,20-Aug,20-Sep,20-Oct,20-Nov,20-Dec,21-Jan,21-Feb,21-Mar,21-Apr,...,22-May,22-Jun,22-Jul,22-Aug,22-Sep,22-Oct,22-Nov,22-Dec,23-Jan,23-Feb
0,"Montréal, Quebec",5993137,6106531,5972435,6209757,5703922,5092495,6101555,6591588,6671994,...,7011267,6946357,6911373,7114241,6967176,7349779,7239519,7296095,7262899,7302933


In [25]:
sales_montreal = my_transform(mont_data)
sales_montreal['Date'] = sales_montreal['Date'].apply(expand_date)
sales_montreal['Date'] = pd.to_datetime(sales_montreal['Date']) # convert date to datetime64[ns] 
sales_montreal.head()

,Date,Value
1,2020-08-01,5993137
2,2020-09-01,6106531
3,2020-10-01,5972435
4,2020-11-01,6209757
5,2020-12-01,5703922


In [26]:
alt.Chart(sales_montreal).mark_line().encode(
    x='Date',
    y='Value'
).interactive()

alt.Chart(...)

### Merge Retail Sales and RT-LBCI

In [27]:
mydfff = pd.merge(RT_LBCI_Montreal_monthly, sales_montreal, on='Date')

#rename columns
mydfff = mydfff.rename(columns={'Value_x': 'RT_LBCI_Montreal_monthly', 'Value_y': 'Retail_Sales'})
mydfff['Retail_Sales'] = mydfff['Retail_Sales'].values.astype(np.float64)
mydfff.head()

,Date,RT_LBCI_Montreal_monthly,Retail_Sales
0,2020-08-01,93.6225,5993137.0
1,2020-09-01,107.2300,6106531.0
2,2020-10-01,120.5750,5972435.0
3,2020-11-01,143.8020,6209757.0
4,2020-12-01,155.1450,5703922.0


In [28]:
# combine the plot of the two dataframes
chart1111 = alt.Chart(mydfff, title = "Montreal Real-time Local Business Condition Index & Retail Sales (Monthly)").mark_line().encode(
    x='Date:T',
    y='RT_LBCI_Montreal_monthly:Q',
)

chart2222 = alt.Chart(mydfff).mark_line(color='red').encode(
    x='Date:T',
    y='Retail_Sales:Q',
)

(chart1111 + chart2222
).properties(width=800,height=500
).resolve_scale(y='independent'
).configure_axis(grid=False
).interactive()

alt.LayerChart(...)

### Correlation between RT-LBCI and Retail Sales

In [29]:
# drop date
nodate_mydfff = mydfff.drop(['Date'], axis=1)

# normalize the data
scaled_mydfff = (nodate_mydfff - nodate_mydfff.min()) / (nodate_mydfff.max() - nodate_mydfff.min())
scaled_mydfff.head()

,RT_LBCI_Montreal_monthly,Retail_Sales
0,0.000000,0.398994
1,0.071446,0.449228
2,0.141513,0.389822
3,0.263465,0.494959
4,0.323021,0.270868


In [30]:
# find_correlation
find_correlation(mydfff, 'RT_LBCI_Montreal_monthly', 'Retail_Sales')

0.7672177578696368

There is a strong correlation between Retail sales and RTLBCI.